In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

In [ ]:
# อ่านข้อมูล metadata 
site_df = pd.read_csv('../data/DeDe_49stations/DeDe_49sites_metadata.csv')

# อ่านข้อมูล radiation 
ssrd_ds = xr.open_dataset('../data/ERA5/radiation/era_ssrd_2023.nc')
str_ds = xr.open_dataset('../data/ERA5/radiation/STR.nc')
ssr_ds = xr.open_dataset('../data/ERA5/radiation/SSR.nc')

In [ ]:
# เก็บข้อมูล radiation ของแต่ละ site ในปี 2023
for i, row in site_df.iterrows():

    # ดึงข้อมูลที่พิกัดละติจูด/ลองจิจูดที่ใกล้ที่สุดจากแต่ละ dataset
    ssrd_df = ssrd_ds.sel(latitude=row['lat'],longitude=row['lng'], method='nearest').to_dataframe()
    str_df = str_ds.sel(latitude=row['lat'],longitude=row['lng'], method='nearest').to_dataframe()
    ssr_df = ssr_ds.sel(latitude=row['lat'],longitude=row['lng'], method='nearest').to_dataframe()
    
    # รวมข้อมูลทุกตัวแปรของ radiation ไว้ใน DataFrame
    df = pd.concat([ssrd_df['ssrd'], str_df['str'], ssr_df['ssr']], axis=1)

    # เเปลง timezone จาก UTC เป็น (Asia/Bangkok)
    df = df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)

    # คัดเลือกเฉพาะข้อมูลในปี 2023
    df = df[df.index.year == 2023]

    # แปลง datetime index เป็นคอลัมน์ และเปลี่ยนชื่อเป็น Datetime
    df_1h = df.reset_index()
    df_1h.rename(columns={'valid_time': 'Datetime'}, inplace=True)

    # โหลดข้อมูลรายชั่โมงเเบบ CSV
    df_1h.to_csv(f'../data/ERA5/radiation/all_site_1hr/era5_1hr_2023_site{row['Site_id']:02d}.csv', index=False)

    # ทำการ resample ข้อมูลเป็น 15 นาที โดยใช้ interpolation
    df = df.resample('15min').asfreq()
    df['ssrd'] = df['ssrd'].interpolate(method='linear')
    df['str'] = df['str'].interpolate(method='linear')
    df['ssr'] = df['ssr'].interpolate(method='linear')

    # แปลง datetime index เป็นคอลัมน์ และเปลี่ยนชื่อเป็น Datetime
    df.reset_index(inplace=True)
    df.rename(columns={'valid_time': 'Datetime'}, inplace=True)

    # โหลดข้อมูลราย 15 นาทีเเบบ CSV
    df.to_csv(f'../data/ERA5/radiation/all_site_15min/era5_15min_2023_site{row['Site_id']:02d}.csv', index=False)


In [ ]:
# อ่านข้อมูล cloud cover ของ ERA5
cloud_ds1 = xr.open_dataset('../data/ERA5/cloud/data_era5_clound_1_2023.nc')
cloud_ds2 = xr.open_dataset('../data/ERA5/cloud/data_era5_clound_2_2023.nc')
cloud_ds3 = xr.open_dataset('../data/ERA5/cloud/data_era5_clound_3_2023.nc')

In [ ]:
# เก็บข้อมูล cloud ของแต่ละ site ในปี 2023
for i, row in site_df.iterrows():

    # ดึงข้อมูลที่พิกัดละติจูด/ลองจิจูดที่ใกล้ที่สุดจากแต่ละ dataset
    df1 = cloud_ds1.sel(latitude=row['lat'], longitude=row['lng'], method='nearest').to_dataframe()
    df2 = cloud_ds2.sel(latitude=row['lat'], longitude=row['lng'], method='nearest').to_dataframe()
    df3 = cloud_ds3.sel(latitude=row['lat'], longitude=row['lng'], method='nearest').to_dataframe()

    # รวมข้อมูลทุกตัวแปรของ cloud ไว้ใน DataFrame
    df = pd.concat([df2['lcc'], df2['mcc'], df1['hcc'], df3['tcc']], axis=1)

    # คัดเลือกเฉพาะข้อมูลในปี 2023
    df = df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)

    # ทำการ resample ข้อมูลเป็นทุก ๆ 15 นาที โดยใช้ interpolation
    df = df.resample('15min').asfreq()
    df = df.interpolate(method='linear')

    # แปลง datetime index เป็นคอลัมน์ และเปลี่ยนชื่อเป็น Datetime
    df.reset_index(inplace=True)
    df.rename(columns={'valid_time':'Datetime'},inplace=True)

    # โหลดข้อมูลราย 15 นาทีเเบบ CSV
    df.to_csv(f'../data/ERA5/cloud/all_site_15min/era5_cloud_15min_2023_site{row['Site_id']:02d}.csv', index=False)
